In [1]:
def get_row_col_range(data, grid, lat_range, lon_range):
    """ Get the row col range given lat lon range.  """
    
    if grid == 'gridT':
        # Get all lat-lon data
        lat = data['nav_lat_grid_T'][:]
        lon = data['nav_lon_grid_T'][:]

    else:
        # Get all lat-lon data
        lat = data['nav_lat'][:]
        lon = data['nav_lon'][:]
        
    # Create mask given lat lon values.
    lat_mask = np.ma.filled((lat.data >= lat_range[0]) & (lat.data <= lat_range[1]))
    lon_mask = np.ma.filled((lon.data >= lon_range[0]) & (lon.data <= lon_range[1]))

    # Apply masks to data
    mask = lat
    mask[~(lat_mask & lon_mask)] = 0

    # Find the row,col range by collapsing each axis.
    row_ranges = np.where(mask.data.sum(axis=1) > 0)[0]
    col_ranges = np.where(mask.data.sum(axis=0) > 0)[0]

    # Select range
    row_range = (row_ranges[0], row_ranges[-1]+1)
    col_range = (col_ranges[0], col_ranges[-1]+1)

    return row_range, col_range


In [ ]:
def getVar_region(run, grid, depth, lon_range,lat_range,var,years_list=None, month_list=None, one_per_month=False,monthly_mean=False,masked=True,cardinal=True):

    """
    
    Extract a variable in an pre-defined area from ANHA4 domain.
    
    Inputs:
    run = Simulation (i.e. ANHA4-WJM004); type: string
    grid = Variable model grid (i.e. griT, gridB, icemod); type: string
    depth = Number of depth levels, including surface; type: integer
    lon_range = A list with west and east region limits; type: integer/float
    lat_range = A list with south and north region limits; type: integer/float
    var = Variable name (i.e. votemper, vosaline); type: string
    years_list = List made of all years of interest; type: string
    month_list = List made of all years of interest; type: string
    one_per_month = If True, extract just the first day of each month
    monthly_mean = If True, ask the user to input the monthly mean directory, since this is not
                   a standard PORTAL output format.
    
    """
    
    # Get mask path
    data_path, mask_path = get_paths(run=run)
    
    # Get file names list
    file_list = get_file_list(run=run,grid=grid, years_list=years_list, month_list=month_list, one_per_month=one_per_month, monthly_mean=monthly_mean)
    
    # Get depth levels
    depth_levels = nc.Dataset(file_list[0])['deptht'][:depth]
    
    # Get latitude and longitude limits and indices
    if cardinal:
        row_range, col_range = get_row_col_range(nc.Dataset(file_list[0]), lat_range, lon_range)
    else:
        row_range, col_range = lat_range, lon_range

    # Get the region land/ocean mask
    mask_region = getMask_region(run,row_range,col_range)

    # For tracers
    if grid == 'gridT':
        
        # Get latitude and longitude
        lon = nc.Dataset(file_list[0])['nav_lon_grid_T'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        lat = nc.Dataset(file_list[0])['nav_lat_grid_T'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        
        # Prealocating variable
        var_region = np.empty([1, depth, np.size(lat,0),np.size(lon,1)])
        
        # Extracting data
        for filename in file_list:
            ds = nc.Dataset(filename)
            ds_var = np.squeeze(ds[var][:,:depth,row_range[0]:row_range[1],col_range[0]:col_range[1]])
            ds_var[mask_region == 0] = np.nan
            var_region = np.append(var_region,ds_var.data,axis=0)
                                              
        
        # Remove empty first time dimension
        var_sec = np.squeeze(var_sec[1:,:,:,:])
        
    # For velocities
    elif any([g in grid for g in ['gridU','gridV','gridW']]):
    
        # Pre alocating variable
        var_data = np.empty([1, depth, np.size(lat,0),np.size(lon,1)])
        
        # Get latitude and longitude
        lon = nc.Dataset(file_list[0])['nav_lon'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        lat = nc.Dataset(file_list[0])['nav_lat'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        
        # Extracting data
        for filename in file_list:
            ds = nc.Dataset(filename)
            ds_var = ds[var][:,:depth,row_range[0]:row_range[1],col_range[0]:col_range[1]]
            ds_var[mask_sect == 0] = np.nan
            var_sec = np.append(var_data,ds_var.data,axis=0)
        
        # Remove empty first time dimension
        var_sec = var_sec[1:,:,:,:]
        
    else:
        # Get latitude and longitude
        lon = nc.Dataset(file_list[0])['nav_lon'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        lat = nc.Dataset(file_list[0])['nav_lat'][row_range[0]:row_range[1],col_range[0]:col_range[1]]
        
        # Prealocating variable
        var_region = np.empty([1, np.size(lat,0),np.size(lon,1)])

        # Extracting data
        for filename in file_list:
            ds = nc.Dataset(filename)
            ds_var = np.squeeze(ds[var][:,row_range[0]:row_range[1],col_range[0]:col_range[1]])
            ds_var[mask_region[0,:,:] == 0] = np.nan
            var_region = np.append(var_region,ds_var.data,axis=0)
                                              
        
        # Remove empty first time dimension
        var_sec = var_sec[1:,:,:]
        
    return var_region, lon, lat, depth_levels